In [1]:
from os.path import join
import pandas as pd
import gc
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader
from catalyst import dl
from catalyst.dl import SupervisedRunner


from dataset import MelanomaDataset, transforms_train, transforms_valid

INPUT_DIR = '../input/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/detkov/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject



device(type='cuda', index=0)

In [2]:
BATCH_SIZE = 32
NUM_WORKERS = 4

submission_df = pd.read_csv(join(INPUT_DIR + 'sample_submission.csv'))
train_df = pd.read_csv(join(INPUT_DIR, 'train_folds.csv'))

dataset_train = MelanomaDataset(train_df[train_df['kfold'].isin([0,1,2,3])], INPUT_DIR)
dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

dataset_val = MelanomaDataset(train_df[train_df['kfold'].isin([4])], INPUT_DIR)
dataloader_val = DataLoader(dataset_train, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False)

loaders = {"train": dataloader_train, "valid": dataloader_val}

In [3]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=True, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

criterion = {'focal': FocalLoss()}

In [8]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.max = nn.AdaptiveMaxPool2d(output_size=(1, 1))

    def forward(self, x):
        avg_x = self.avg(x)
        max_x = self.max(x)
        return torch.cat([avg_x, max_x], dim=1)


class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)


class Model(nn.Module):
    
    def __init__(self, c_out=1):
        super().__init__()
        m = models.resnet18(pretrained=True)
            
        c_feature = list(m.children())[-1].in_features
        self.base = nn.Sequential(*list(m.children())[:-2])
        self.head = nn.Sequential(AdaptiveConcatPool2d(),
                                  Flatten(),
                                  nn.Linear(c_feature * 2, c_out))

        
    def forward(self, x):
        x = x.permute(0, 3, 1, 2).float()
        h = self.base(x)
        logits = self.head(h).squeeze(1)
        return logits

model = Model()

# import pretrainedmodels


# def get_model(model_name: str, num_classes: int, pretrained: str = "imagenet"):
#     model_fn = pretrainedmodels.__dict__[model_name]
#     model = model_fn(num_classes=1000, pretrained=pretrained)
    
#     model.fc = nn.Sequential()
#     dim_feats = model.last_linear.in_features
#     model.last_linear = nn.Linear(dim_feats, num_classes)

#     return model


# model_name = "resnet34"
# model = get_model(model_name, 1)

In [9]:
LR = 3e-4
EPOCHS = 30

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=3e-6)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=10 * LR, 
                                                epochs=EPOCHS, steps_per_epoch=len(dataloader_train))

In [10]:
runner = SupervisedRunner(device=device)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
#     scheduler=scheduler,
    loaders=loaders,
    callbacks=[dl.callbacks.metrics.auc.AUCCallback()],
    logdir="./logs",
    num_epochs=EPOCHS,
    main_metric='auc',
    verbose=True,
    fp16=False,
    timeit=True
)


1/30 * Epoch (train):   0% 0/746 [00:00<?, ?it/s]

IndexError: too many indices for tensor of dimension 1

In [11]:
torch.cuda.empty_cache()
gc.collect()

27

In [ ]:
# class EnsembleModel:
#     def __init__(self, models):
#         self.models = models
    
#     def __call__(self, x):
#         res = []
#         x = x.cuda()
#         with torch.no_grad():
#             for m in self.models:
#                 res.append(m(x))
#         res = torch.stack(res)
#         return torch.mean(res, dim=0)